In [1]:
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
library(parallel)
library(reshape2)
library(ggplot2)
library(patchwork)
library(ggpubr)
library(rstatix)
source('../../helper_function/helper_function_IHA.r')

Warning message:
“package ‘plyr’ was built under R version 4.3.3”
Warning message:
“package ‘dplyr’ was built under R version 4.3.3”
Warning message:
“package ‘tidyr’ was built under R version 4.3.3”
Warning message:
“package ‘tibble’ was built under R version 4.3.3”
Warning message:
“package ‘reshape2’ was built under R version 4.3.3”

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message:
“package ‘patchwork’ was built under R version 4.3.3”

Attaching package: ‘ggpubr’


The following object is masked from ‘package:plyr’:

    mutate


Warning message:
“package ‘rstatix’ was built under R version 4.3.3”

Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter




In [3]:
meta_data=read.csv("../../Dataset/SF4_meta_data-2024-05-05.csv")
meta_data$ageAtEnrollment[meta_data$ageAtEnrollment == '89+'] <- '90'
meta_data$ageAtEnrollment<-as.numeric(meta_data$ageAtEnrollment)
meta_data<-meta_data[!is.na(meta_data$ageAtEnrollment),]
meta_data$Ages<-meta_data$ageAtEnrollment
#meta_data <- meta_data[!duplicated(meta_data$sample.sampleKitGuid, fromLast = TRUE), ]

In [4]:
table(meta_data$lab.revisionHistory.dataHistory.CMV.Ab.Screen.Result..CMVRES..newValue)


Negative Positive 
      98      136 

In [5]:
count_data<-read.csv("SF4_AIFI_L3_Counts.csv")

In [5]:
count_data <- count_data %>%
  dplyr::group_by(pbmc_sample_id) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
  dplyr::group_by(pbmc_sample_id) %>%     
  dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% 
  dplyr::ungroup() %>% as.data.frame(.)

head(count_data)

,X,celltypist_l3,count,pbmc_sample_id,percentage,percentage_clr
,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>
1,0,GZMK- CD56dim NK cell,2696,PB03173-001,11.288837,3.787325
2,1,Core naive B cell,2351,PB03173-001,9.844234,3.650396
3,2,CM CD4 T cell,2341,PB03173-001,9.802362,3.646134
4,3,GZMB- CD27+ EM CD4 T cell,2202,PB03173-001,9.220333,3.584922
5,4,GZMB- CD27- EM CD4 T cell,2030,PB03173-001,8.500126,3.503591
6,5,GZMK+ Vd2 gdT,1401,PB03173-001,5.866343,3.132742


In [6]:
meta_data<-meta_data %>% filter(lab.revisionHistory.dataHistory.CMV.Ab.Screen.Result..CMVRES..newValue%in% c("Negative",'Positive'))

In [7]:
meta_data$CMV<-meta_data$lab.revisionHistory.dataHistory.CMV.Ab.Screen.Result..CMVRES..newValue

In [8]:
count_data<-count_data %>% filter(pbmc_sample_id %in% meta_data$pbmc_sample_id)

In [9]:
count_data<-left_join(count_data,meta_data,by=c("pbmc_sample_id"))

In [10]:
count_data<-count_data %>% group_by(sample.sampleKitGuid) %>%
  dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
  ungroup()%>% 
  group_by(sample.sampleKitGuid) %>%     
  dplyr::mutate(AIFI_L3_clr = clr_transform((percentage))) %>%
  dplyr::ungroup()

In [11]:
log2fc<-count_data %>%
  select(percentage,AIFI_L3_clr, celltypist_l3, cohort.cohortGuid, subject.biologicalSex, CMV) %>%
  group_by(celltypist_l3, CMV) %>%
  summarise(mean_AIFI_L3_clr = median(AIFI_L3_clr, na.rm = TRUE)) %>%
  spread(CMV, mean_AIFI_L3_clr) %>%
  mutate(log2fc = log2(`Positive` / `Negative`)) %>%
  mutate(delta_change = `Positive` - `Negative`) 

`summarise()` has grouped output by 'celltypist_l3'. You can override using the
`.groups` argument.
Warning message:
“There were 7 warnings in `mutate()`.
The first warning was:
ℹ In argument: `log2fc = log2(Positive/Negative)`.
ℹ In group 3: `celltypist_l3 = "Adaptive NK cell"`.
Caused by warning:
! NaNs produced
ℹ Run `dplyr::last_dplyr_warnings()` to see the 6 remaining warnings.”


In [12]:
STAT<-count_data %>%
  select(AIFI_L3_clr,celltypist_l3, cohort.cohortGuid,subject.biologicalSex,CMV) %>%
  group_by(celltypist_l3)  %>%
  wilcox_test(AIFI_L3_clr ~ CMV
             ) %>%
  adjust_pvalue(method='BH') %>%
  add_significance() %>% as.data.frame()

In [13]:
STAT<-left_join(STAT,log2fc)

Joining with `by = join_by(celltypist_l3)`


In [14]:
colnames(STAT)[1]<-"AIFI_L3"

# Get DEG Number

In [15]:
DEG_res<-read.csv("Deseq2_Result_SF4.csv")

In [16]:
DEG_res_signi<-DEG_res %>% filter(padj<0.05,abs(log2FoldChange)>0.1)

In [17]:
DEG_counts<-as.data.frame(table(DEG_res_signi$contrast,DEG_res_signi$celltype))

In [18]:
colnames(DEG_counts)<-c("factor","AIFI_L3","DEG_Counts")

# Combine STAT


In [19]:
STAT<-left_join(STAT,DEG_counts )

Joining with `by = join_by(AIFI_L3)`


In [20]:
STAT<-STAT%>%
  mutate(adjP_degs_combined_value = if_else(DEG_Counts != 0,
                                            -log10(p.adj)  * DEG_Counts * 0.1,
                                            -log10(p.adj)  * 0.1))

In [21]:
write.csv(STAT,"DEG_counts_and_Frequency_Changes.csv")